In [ ]:
import pandas as pd
import numpy as np
import os
import gc

import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# set a seed value
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score

!pip install sentencepiece
!pip install transformers
import transformers
from transformers import AdamW

import warnings
warnings.filterwarnings("ignore")


print(torch.__version__)

# Section 2

## 2.1. Load the Data

In [ ]:
sentiment_to_num = {
    'Negative': 0,
    'Neutral': 1,
    'Positive': 2
}

In [ ]:
sampled_df = pd.read_csv(r'sampled_data.csv', sep='\t', encoding='ISO-8859-1')

df_train = sampled_df.groupby('sentiment').sample(n=40000).sample(frac=1)
df_test = sampled_df.filter(items=list(filter(lambda x: x not in df_train.index, sampled_df.index)), axis=0)
df_train = df_train.reset_index()[['text', 'sentiment']]
df_test = df_test.reset_index()[['text', 'sentiment']]
df_train['sentiment'] =  df_train['sentiment'].apply(lambda x: sentiment_to_num[x])
df_test['sentiment'] =  df_test['sentiment'].apply(lambda x: sentiment_to_num[x])
df_test

## 2.2. Create Folds

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold

# shuffle
df = shuffle(df_train)

# initialize kfold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1024)

# for stratification
y = df['sentiment']

# Note:
# Each fold is a tuple ([train_index_values], [val_index_values])
# fold_0, fold_1, fold_2, fold_3, fold_5 = kf.split(df, y)

# Put the folds into a list. This is a list of tuples.
fold_list = list(kf.split(df, y))

train_df_list = []
val_df_list = []

for i, fold in enumerate(fold_list):

    # map the train and val index values to dataframe rows
    df_train = df[df.index.isin(fold[0])]
    df_val = df[df.index.isin(fold[1])]
    
    train_df_list.append(df_train)
    val_df_list.append(df_val)
    
    

print(len(train_df_list))
print(len(val_df_list))

# Section 3

## 3.1. Train a Model

In [ ]:
MODEL_TYPE = 'bert-base-multilingual-uncased'

NUM_FOLDS = 5

# Saving 5 TPU models will exceed the 4.9GB disk space.
# Therefore, will will only train on 3 folds.
NUM_FOLDS_TO_TRAIN = 5 

L_RATE = 2e-5
MAX_LEN = 256
NUM_EPOCHS = 100
BATCH_SIZE = 64
NUM_CORES = os.cpu_count()

NUM_CORES

## Instantiate the tokenizer

In [ ]:
from transformers import BertTokenizer

# Load the tokenizer.
print('Loading tokenizer...')
tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE, do_lower_case=True)

## Create the Dataloader

In [ ]:
from language_tokens import get_lang_tokens

In [ ]:
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df



    def __getitem__(self, index):

        # get the sentence from the dataframe
        features = self.df_data.loc[index, 'text']

        # Process the sentence
        # ---------------------

        encoded_dict = tokenizer.encode_plus(
                    features,           # Sentences to encode.
                    add_special_tokens = True,      # Add '[CLS]' and '[SEP]'
                    truncation = True,
                    max_length = MAX_LEN,           # Pad or truncate all sentences.
                    pad_to_max_length = True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',          # Return pytorch tensors.
               )  
        
        # These are torch tensors already.
        input_ids = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        token_type_ids = encoded_dict['token_type_ids'][0]
        language_ids = torch.tensor(get_lang_tokens(
            [x.replace(' ', '') for x in tokenizer.batch_decode(input_ids.tolist())]
            ))
        
        # Convert the target to a torch tensor
        target = torch.tensor(self.df_data.loc[index, 'sentiment'])

        sample = (input_ids, att_mask, token_type_ids, language_ids, target)


        return sample


    def __len__(self):
        return len(self.df_data)
    
    
    
    
    

class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df



    def __getitem__(self, index):

        # get the sentence from the dataframe
        features = self.df_data.loc[index, 'text']

        # Process the sentence
        # ---------------------

        encoded_dict = tokenizer.encode_plus(
                    features,           # Sentence to encode.
                    add_special_tokens = True,      # Add '[CLS]' and '[SEP]'
                    truncation = True,
                    max_length = MAX_LEN,           # Pad or truncate all sentences.
                    pad_to_max_length = True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',          # Return pytorch tensors.
               )
        
        # These are torch tensors already.
        input_ids = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        token_type_ids = encoded_dict['token_type_ids'][0]
        language_ids = torch.tensor(get_lang_tokens(
            [x.replace(' ', '') for x in tokenizer.batch_decode(input_ids.tolist())]
            ))
               

        sample = (input_ids, att_mask, token_type_ids, language_ids)


        return sample


    def __len__(self):
        return len(self.df_data)



## Test the dataloader

In [ ]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

df_val

In [ ]:
train_data = CompDataset(df_train)
val_data = CompDataset(df_val)
test_data = TestDataset(df_test)



train_dataloader = torch.utils.data.DataLoader(train_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                       num_workers=NUM_CORES)

val_dataloader = torch.utils.data.DataLoader(val_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                       num_workers=NUM_CORES)

test_dataloader = torch.utils.data.DataLoader(test_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False,
                                       num_workers=NUM_CORES)



print(len(train_dataloader))
print(len(val_dataloader))
print(len(test_dataloader))

input_ids, att_mask, token_type_ids, language_ids, target = next(iter(train_dataloader))

## Define the Model

In [ ]:
import torch

# check if we have cuda installed
if torch.cuda.is_available():
    # to use GPU
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('GPU is:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
from transformers import BertConfig
from modeling_bert import BertForSequenceClassification

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
config = BertConfig.from_pretrained(
    MODEL_TYPE,
    num_labels = len(set(target)),
    output_attentions = False,
    output_hidden_states = False,
    num_hidden_layers = 5,
    num_attention_heads = 8,
    hidden_dropout_prob = 0.2,
    attention_probs_dropout_prob = 0.2,
    ignore_mismatched_sizes=True
    )

model = BertForSequenceClassification.from_pretrained(
    MODEL_TYPE,
    config=config
)

# Send the model to the device.
model.to(device)

In [ ]:
optimizer = AdamW(model.parameters(),
  lr = L_RATE, 
  eps = 1e-8
)

## Train the Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# with open ('/content/drive/MyDrive/model.bin', 'rb') as fp:
#   states = torch.load(fp)

#   curr_epoch = states['epoch']
#   model.load_state_dict(states['model'])
#   optimizer.load_state_dict(states['optimizer'])
#   fold_val_acc_list = states['losses']

#   del states

# curr_epoch

In [ ]:
# initial settings

curr_epoch = 0

# Store the accuracy scores for each fold model in this list.
# [[model_0 scores], [model_1 scores], [model_2 scores], [model_3 scores], [model_4 scores]]
# [[ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...]]

# Create a list of lists to store the val acc results.
# The number of items in this list will correspond to
# the number of folds that the model is being trained on.
fold_val_acc_list = []

for i in range(0, NUM_FOLDS):
    
    # append an empty list
    fold_val_acc_list.append([])

In [ ]:
%%time

import pickle
from tqdm.auto import tqdm

# Set a seed value.
seed_val = 1024

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
    

# For each epoch...
for epoch in range(curr_epoch, NUM_EPOCHS):
    
    print("\nNum folds used for training:", NUM_FOLDS_TO_TRAIN)
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    
    # Get the number of folds
    num_folds = len(train_df_list)

    # For this epoch, store the val acc scores for each fold in this list.
    # We will use this list to calculate the cv at the end of the epoch.
    epoch_acc_scores_list = []
    
    # For each fold...
    for fold_index in range(1, NUM_FOLDS_TO_TRAIN):
        
        print('\n== Fold Model', fold_index)

        # ========================================
        #               Training
        # ========================================
        
        stacked_val_labels = []
        targets_list = []

        print('Training...')

        progress_bar_train =  tqdm(range(len(train_dataloader)))

        # put the model into train mode
        model.train()

        # This turns gradient calculations on and off.
        torch.set_grad_enabled(True)


        # Reset the total loss for this epoch.
        total_train_loss = 0

        for i, batch in enumerate(train_dataloader):

            train_status = 'Batch ' + str(i+1) + ' of ' + str(len(train_dataloader))

            print(train_status, end='\r')


            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_token_type_ids = batch[2].to(device)
            b_language_ids = batch[3].to(device)
            b_labels = batch[4].to(device)

            model.zero_grad()        


            outputs = model(b_input_ids, 
                        token_type_ids=b_token_type_ids, 
                        attention_mask=b_input_mask,
                        language_ids=b_language_ids,
                        labels=b_labels)
            
            progress_bar_train.update(1)

            # Get the loss from the outputs tuple: (loss, logits)
            loss = outputs[0]

            # Convert the loss from a torch tensor to a number.
            # Calculate the total loss.
            total_train_loss = total_train_loss + loss.item()

            # Zero the gradients
            optimizer.zero_grad()

            # Perform a backward pass to calculate the gradients.
            loss.backward()
            
            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Use the optimizer to update Weights
            
            # Optimizer for GPU
            optimizer.step() 
            

        print('Train loss:' ,total_train_loss)


        # ========================================
        #               Validation
        # ========================================

        print('\nValidation...')

        progress_bar_eval =  tqdm(range(len(val_dataloader)))

        # Put the model in evaluation mode.
        model.eval()

        # Turn off the gradient calculations.
        # This tells the model not to compute or store gradients.
        # This step saves memory and speeds up validation.
        torch.set_grad_enabled(False)


        # Reset the total loss for this epoch.
        total_val_loss = 0


        for j, val_batch in enumerate(val_dataloader):

            val_status = 'Batch ' + str(j+1) + ' of ' + str(len(val_dataloader))

            print(val_status, end='\r')

            b_input_ids = val_batch[0].to(device)
            b_input_mask = val_batch[1].to(device)
            b_token_type_ids = val_batch[2].to(device)
            b_language_ids = batch[3].to(device)
            b_labels = val_batch[4].to(device)      


            outputs = model(b_input_ids, 
                    token_type_ids=b_token_type_ids, 
                    attention_mask=b_input_mask, 
                    language_ids=b_language_ids,
                    labels=b_labels)
            
            progress_bar_eval.update(1)

            # Get the loss from the outputs tuple: (loss, logits)
            loss = outputs[0]

            # Convert the loss from a torch tensor to a number.
            # Calculate the total loss.
            total_val_loss = total_val_loss + loss.item()

            # Get the preds
            preds = outputs[1]


            # Move preds to the CPU
            val_preds = preds.detach().cpu().numpy()

            # Move the labels to the cpu
            targets_np = b_labels.to('cpu').numpy()

            # Append the labels to a numpy list
            targets_list.extend(targets_np)

            if j == 0:  # first batch
                stacked_val_preds = val_preds

            else:
                stacked_val_preds = np.vstack((stacked_val_preds, val_preds))
                
                
        # .........................................
        # Calculate the val accuracy for this fold
        # .........................................      


        # Calculate the validation accuracy
        y_true = targets_list
        y_pred = np.argmax(stacked_val_preds, axis=1)

        val_acc = accuracy_score(y_true, y_pred)
        
        
        epoch_acc_scores_list.append(val_acc)


        print('Val loss:' ,total_val_loss)
        print('Val acc: ', val_acc)
        
        
        # .........................
        # Save the best model
        # .........................
        
        if epoch == 0:
            
            # Save the Model
            torch.save({
                'epoch': epoch,
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'losses': fold_val_acc_list
            }, '/content/drive/MyDrive/model.bin')
            print('Saved model.')
            
        if epoch != 0:
        
            val_acc_list = fold_val_acc_list[fold_index]
            best_val_acc = max(val_acc_list)
            
            if val_acc > best_val_acc:
                # save the model
                torch.save({
                    'epoch': epoch,
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'losses': fold_val_acc_list
                }, '/content/drive/MyDrive/model.bin')
                print('Val acc improved. Saved model')
                
                
        # .....................................
        # Save the val_acc for this fold model
        # .....................................
        
        # Note: Don't do this before the above 'Save Model' code or 
        # the save model code won't work. This is because the best_val_acc will
        # become current val accuracy.
                
        # fold_val_acc_list is a list of lists.
        # Each fold model has it's own list corresponding to the fold index.
        # Here we choose a list corresponding to the fold number and append the acc score to that list.
        fold_val_acc_list[fold_index].append(val_acc)
        
            

        # Use the garbage collector to save memory.
        gc.collect()
        
    # .............................................................
    # Calculate the CV accuracy score over all folds in this epoch
    # .............................................................   
        
        
    # Print the average val accuracy for all 5 folds
    cv_acc = sum(epoch_acc_scores_list)/NUM_FOLDS_TO_TRAIN
    print("\nCV Acc:", cv_acc)

In [ ]:
model.save('/content/drive/MyDrive/final_model')

## Make a Test Set Prediction

In [ ]:
# ========================================
#               Test Set
# ========================================

print('\nTest Set...')

model_preds_list = []

print('Total batches:', len(test_dataloader))

for fold_index in range(0, NUM_FOLDS_TO_TRAIN):

    progress_bar_test =  tqdm(range(len(test_dataloader)))
    
    # print('\nFold Model', fold_index)

    # # Load the fold model
    # path_model = 'model_' + str(fold_index) + '.bin'
    # model.load_state_dict(torch.load(path_model))

    # # Send the model to the device.
    # model.to(device)

    stacked_val_labels = []
    

    # Put the model in evaluation mode.
    model.eval()

    # Turn off the gradient calculations.
    # This tells the model not to compute or store gradients.
    # This step saves memory and speeds up validation.
    torch.set_grad_enabled(False)


    # Reset the total loss for this epoch.
    total_val_loss = 0

    for j, h_batch in enumerate(test_dataloader):

        inference_status = 'Batch ' + str(j + 1)

        print(inference_status, end='\r')

        b_input_ids = h_batch[0].to(device)
        b_input_mask = h_batch[1].to(device)
        b_token_type_ids = h_batch[2].to(device)  
        b_language_ids = h_batch[3].to(device) 


        outputs = model(b_input_ids, 
                token_type_ids=b_token_type_ids, 
                language_ids=b_language_ids,
                attention_mask=b_input_mask)
        
        progress_bar_test.update(1)


        # Get the preds
        preds = outputs[0]


        # Move preds to the CPU
        val_preds = preds.detach().cpu().numpy()
        
        
        # Stack the predictions.

        if j == 0:  # first batch
            stacked_val_preds = val_preds

        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))

        
    model_preds_list.append(stacked_val_preds)
    
            
print('\nPrediction complete.')        

## Process the predictions

Here we are ensembling the predictons of all the fold models. We add all the matrices and then take the average.

In [ ]:
# Sum the predictions of all fold models
for i, item in enumerate(model_preds_list):
    
    if i == 0:
        
        preds = item
        
    else:
    
        # Sum the matrices
        preds = item + preds

        
# Average the predictions
avg_preds = preds/(len(model_preds_list))


test_preds = np.argmax(avg_preds, axis=1)

In [ ]:
import numpy as np

correct = (test_preds == np.array(list(df_test.sentiment)))
accuracy = correct.sum()

accuracy / len(test_preds)